In [ ]:
!pip install datasets transformers trl evaluate huggingface_hub pynvml psutil rouge_score trackio



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.0/875.0 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 124.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=67804b6f2234cb3fb392360cd362db9ef350518a042a6dd38e572f81606c62ee
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [ ]:
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import evaluate
from transformers import TrainerCallback
import time
from huggingface_hub import login
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
import pynvml
import psutil
from peft import AdaLoraConfig, get_peft_model
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

torch.cuda.empty_cache()
print("login")

login(token="YourToken")

print("data is loading")

ds_train = load_dataset('csv', data_files='./drive/MyDrive/text_only_train_data.csv')
ds_val = load_dataset('csv', data_files={'validation':'./drive/MyDrive/text_only_val_data.csv'})

train = ds_train["train"]

val = ds_val["validation"]

# Optionally add prefix on the validation set for instruction tuning
def add_prefix(example):
    prefix = "Summarize the following document: "
    example["document"] = prefix + example["document"]
    return example

train = train.map(add_prefix)
val = val.map(add_prefix)

print("model loading")

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, dtype=torch.bfloat16, device_map="auto")

print("Loading ROUGE metric...")
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # decode token IDs into strings if needed (tokenizer needed here)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Run rouge evaluation on decoded text
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print("ROUGE Log", result)
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }
class ResourceLoggingCallback(TrainerCallback):
    def __init__(self, log_interval=60):
        super().__init__()
        self.log_interval = log_interval  # seconds
        self.last_log_time = time.time()
        nvmlInit()
        self.handle = nvmlDeviceGetHandleByIndex(0)  # GPU 0

    def on_step_end(self, args, state, control, **kwargs):
        current_time = time.time()
        if current_time - self.last_log_time >= self.log_interval:
            # Log GPU memory
            mem_info = nvmlDeviceGetMemoryInfo(self.handle)
            gpu_used = mem_info.used / (1024 ** 3)
            gpu_total = mem_info.total / (1024 ** 3)

            # Log CPU/RAM usage
            cpu_percent = psutil.cpu_percent()
            ram_info = psutil.virtual_memory()

            print(f"[Resource Log | Step {state.global_step}] "
                  f"GPU Memory Used: {gpu_used:.2f} GB / {gpu_total:.2f} GB, "
                  f"CPU Usage: {cpu_percent}%, "
                  f"RAM Usage: {ram_info.used / (1024 ** 3):.2f} GB / {ram_info.total / (1024 ** 3):.2f} GB")

            self.last_log_time = current_time

    def on_train_end(self, args, state, control, **kwargs):
        total_flos = getattr(state, 'total_flos', None)
        if total_flos is not None:
            print(f"Total FLOPs during training: {total_flos}")
        else:
            print("Total FLOPs information not available.")

print(model_id)
print("training")
adalora_config = AdaLoraConfig(
    init_r=64,
    target_r = 32,# rank for LoRA matrices
    target_modules=["q_proj", "v_proj"],  # modules to adapt
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    total_step=471
)

# Wrap the base model with AdaLoRA PEFT model
ada_lora_model = get_peft_model(model, adalora_config)

training_args = SFTConfig(
    dataset_text_field="document",
    learning_rate=2e-4,
    per_device_train_batch_size=1, # Further reduced batch size
    per_device_eval_batch_size=1, # Further reduced eval batch size
    gradient_accumulation_steps=64, # Increased gradient accumulation steps
    num_train_epochs=2,
    report_to=["trackio"],  # Add this to save checkpoints
    save_strategy="epoch",
    output_dir = "./drive/MyDrive/Results_AdaLora/Mistral/sft-mistral-ada-lora",  # Save checkpoint at end of each epoch
    gradient_checkpointing=True # Enable gradient checkpointing
)

trainer = SFTTrainer(
        model=ada_lora_model,  # reload or clone model if needed
        train_dataset=train,
        eval_dataset=val,
        args=training_args,
        compute_metrics=compute_metrics,
        callbacks=[ResourceLoggingCallback(log_interval=120)],
        processing_class=tokenizer,
    )
trainer.train()
trainer.save_model("./drive/MyDrive/Results_AdaLora/Mistral/my_trained_model")
trainer.save_state()

login
data is loading


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16820 [00:00<?, ? examples/s]

Map:   0%|          | 0/2102 [00:00<?, ? examples/s]

...

The model is already on multiple devices. Skipping the move to device specified in `args`.


* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: Dali-bot/trackio-dataset
* Found existing space: https://huggingface.co/spaces/Dali-bot/trackio
* View dashboard by going to: https://Dali-bot-trackio.hf.space/


* Created new run: Dali-bot-1762764337
[Resource Log | Step 1] GPU Memory Used: 15.75 GB / 40.00 GB, CPU Usage: 6.1%, RAM Usage: 5.46 GB / 83.47 GB


Step,Training Loss
10,144.606600
20,95.605600
30,52.900400
40,29.039900
50,21.862300
60,13.017300
70,5.598400
80,4.108400
90,3.001200
100,2.725200


[Resource Log | Step 7] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 5.47 GB / 83.47 GB
[Resource Log | Step 13] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.46 GB / 83.47 GB
[Resource Log | Step 19] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.45 GB / 83.47 GB
[Resource Log | Step 25] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.45 GB / 83.47 GB
[Resource Log | Step 31] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.44 GB / 83.47 GB
[Resource Log | Step 37] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.42 GB / 83.47 GB
[Resource Log | Step 43] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.43 GB / 83.47 GB
[Resource Log | Step 49] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.44 GB / 83.47 GB
[Resource Log | Step 55] GPU Memory Used: 15.85 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.43 GB / 83.47 GB
[R

In [ ]:
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import evaluate
from transformers import TrainerCallback
import time
from huggingface_hub import login
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
import pynvml
import psutil
from peft import AdaLoraConfig, get_peft_model
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

torch.cuda.empty_cache()
print("login")

login(token="YourToken")

print("data is loading")

ds_train = load_dataset('csv', data_files='./drive/MyDrive/text_only_train_data.csv')
ds_val = load_dataset('csv', data_files={'validation':'./drive/MyDrive/text_only_val_data.csv'})

train = ds_train["train"]

val = ds_val["validation"]

# Optionally add prefix on the validation set for instruction tuning
def add_prefix(example):
    prefix = "Summarize the following document: "
    example["document"] = prefix + example["document"]
    return example

train = train.map(add_prefix)
val = val.map(add_prefix)

print("model loading")

model_id = "CohereLabs/c4ai-command-r7b-12-2024"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, dtype=torch.bfloat16, device_map="auto")

print("Loading ROUGE metric...")
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # decode token IDs into strings if needed (tokenizer needed here)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Run rouge evaluation on decoded text
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print("ROUGE Log", result)
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }
class ResourceLoggingCallback(TrainerCallback):
    def __init__(self, log_interval=60):
        super().__init__()
        self.log_interval = log_interval  # seconds
        self.last_log_time = time.time()
        nvmlInit()
        self.handle = nvmlDeviceGetHandleByIndex(0)  # GPU 0

    def on_step_end(self, args, state, control, **kwargs):
        current_time = time.time()
        if current_time - self.last_log_time >= self.log_interval:
            # Log GPU memory
            mem_info = nvmlDeviceGetMemoryInfo(self.handle)
            gpu_used = mem_info.used / (1024 ** 3)
            gpu_total = mem_info.total / (1024 ** 3)

            # Log CPU/RAM usage
            cpu_percent = psutil.cpu_percent()
            ram_info = psutil.virtual_memory()

            print(f"[Resource Log | Step {state.global_step}] "
                  f"GPU Memory Used: {gpu_used:.2f} GB / {gpu_total:.2f} GB, "
                  f"CPU Usage: {cpu_percent}%, "
                  f"RAM Usage: {ram_info.used / (1024 ** 3):.2f} GB / {ram_info.total / (1024 ** 3):.2f} GB")

            self.last_log_time = current_time

    def on_train_end(self, args, state, control, **kwargs):
        total_flos = getattr(state, 'total_flos', None)
        if total_flos is not None:
            print(f"Total FLOPs during training: {total_flos}")
        else:
            print("Total FLOPs information not available.")

print(model_id)
print("training")
adalora_config = AdaLoraConfig(
    init_r=64,
    target_r = 32,# rank for LoRA matrices
    target_modules=["q_proj", "v_proj"],  # modules to adapt
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    total_step=471
)

# Wrap the base model with AdaLoRA PEFT model
ada_lora_model = get_peft_model(model, adalora_config)

training_args = SFTConfig(
    dataset_text_field="document",
    learning_rate=2e-4,
    per_device_train_batch_size=1, # Further reduced batch size
    per_device_eval_batch_size=1, # Further reduced eval batch size
    gradient_accumulation_steps=64, # Increased gradient accumulation steps
    num_train_epochs=2,
    report_to=["trackio"],  # Add this to save checkpoints
    save_strategy="epoch",
    output_dir = "./drive/MyDrive/Results_AdaLora/Command/sft-command-ada-lora",  # Save checkpoint at end of each epoch
    gradient_checkpointing=True # Enable gradient checkpointing
)

trainer = SFTTrainer(
        model=ada_lora_model,  # reload or clone model if needed
        train_dataset=train,
        eval_dataset=val,
        args=training_args,
        compute_metrics=compute_metrics,
        callbacks=[ResourceLoggingCallback(log_interval=120)],
        processing_class=tokenizer,
    )
trainer.train()
trainer.save_model("./drive/MyDrive/Results_AdaLora/Command/my_trained_command_model")
trainer.save_state()

login
data is loading


Map:   0%|          | 0/16820 [00:00<?, ? examples/s]

Map:   0%|          | 0/2102 [00:00<?, ? examples/s]

model loading


tokenizer_config.json:   0%|          | 0.00/45.0k [00:00<?, ?B/s]

...

Truncating eval dataset:   0%|          | 0/2102 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


* Created new run: Dali-bot-1762776522
[Resource Log | Step 1] GPU Memory Used: 20.65 GB / 40.00 GB, CPU Usage: 13.8%, RAM Usage: 5.72 GB / 83.47 GB


Step,Training Loss
10,144.767600
20,95.774300
30,53.062900
40,29.206700
50,22.032100
60,13.208400
70,5.804400
80,4.307700
90,3.208700
100,2.937100


[Resource Log | Step 6] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 5.76 GB / 83.47 GB
[Resource Log | Step 11] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 5.73 GB / 83.47 GB
[Resource Log | Step 16] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.74 GB / 83.47 GB
[Resource Log | Step 21] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.74 GB / 83.47 GB
[Resource Log | Step 26] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.74 GB / 83.47 GB
[Resource Log | Step 31] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.73 GB / 83.47 GB
[Resource Log | Step 36] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.74 GB / 83.47 GB
[Resource Log | Step 41] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 5.71 GB / 83.47 GB
[Resource Log | Step 46] GPU Memory Used: 20.75 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.72 GB / 83.47 GB
[R

In [ ]:
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import evaluate
from transformers import TrainerCallback
import time
from huggingface_hub import login
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
import pynvml
import psutil
from peft import AdaLoraConfig, get_peft_model
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

torch.cuda.empty_cache()
print("login")

login(token="YourToken")

print("data is loading")

ds_train = load_dataset('csv', data_files='./drive/MyDrive/text_only_train_data.csv')
ds_val = load_dataset('csv', data_files={'validation':'./drive/MyDrive/text_only_val_data.csv'})

train = ds_train["train"]

val = ds_val["validation"]

# Optionally add prefix on the validation set for instruction tuning
def add_prefix(example):
    prefix = "Summarize the following document: "
    example["document"] = prefix + example["document"]
    return example

train = train.map(add_prefix)
val = val.map(add_prefix)

print("model loading")

model_id = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, dtype=torch.bfloat16, device_map="auto")

print("Loading ROUGE metric...")
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # decode token IDs into strings if needed (tokenizer needed here)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Run rouge evaluation on decoded text
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print("ROUGE Log", result)
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }
class ResourceLoggingCallback(TrainerCallback):
    def __init__(self, log_interval=60):
        super().__init__()
        self.log_interval = log_interval  # seconds
        self.last_log_time = time.time()
        nvmlInit()
        self.handle = nvmlDeviceGetHandleByIndex(0)  # GPU 0

    def on_step_end(self, args, state, control, **kwargs):
        current_time = time.time()
        if current_time - self.last_log_time >= self.log_interval:
            # Log GPU memory
            mem_info = nvmlDeviceGetMemoryInfo(self.handle)
            gpu_used = mem_info.used / (1024 ** 3)
            gpu_total = mem_info.total / (1024 ** 3)

            # Log CPU/RAM usage
            cpu_percent = psutil.cpu_percent()
            ram_info = psutil.virtual_memory()

            print(f"[Resource Log | Step {state.global_step}] "
                  f"GPU Memory Used: {gpu_used:.2f} GB / {gpu_total:.2f} GB, "
                  f"CPU Usage: {cpu_percent}%, "
                  f"RAM Usage: {ram_info.used / (1024 ** 3):.2f} GB / {ram_info.total / (1024 ** 3):.2f} GB")

            self.last_log_time = current_time

    def on_train_end(self, args, state, control, **kwargs):
        total_flos = getattr(state, 'total_flos', None)
        if total_flos is not None:
            print(f"Total FLOPs during training: {total_flos}")
        else:
            print("Total FLOPs information not available.")

print(model_id)
print("training")
adalora_config = AdaLoraConfig(
    init_r=64,
    target_r = 32,# rank for LoRA matrices
    target_modules=["q_proj", "v_proj"],  # modules to adapt
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    total_step=471
)

# Wrap the base model with AdaLoRA PEFT model
ada_lora_model = get_peft_model(model, adalora_config)

training_args = SFTConfig(
    dataset_text_field="document",
    learning_rate=2e-4,
    per_device_train_batch_size=1, # Further reduced batch size
    per_device_eval_batch_size=1, # Further reduced eval batch size
    gradient_accumulation_steps=64, # Increased gradient accumulation steps
    num_train_epochs=2,
    report_to=["trackio"],  # Add this to save checkpoints
    save_strategy="epoch",
    output_dir = "./drive/MyDrive/Results_AdaLora/Qwen/sft-qwen-ada-lora",  # Save checkpoint at end of each epoch
    gradient_checkpointing=True # Enable gradient checkpointing
)

trainer = SFTTrainer(
        model=ada_lora_model,  # reload or clone model if needed
        train_dataset=train,
        eval_dataset=val,
        args=training_args,
        compute_metrics=compute_metrics,
        callbacks=[ResourceLoggingCallback(log_interval=120)],
        processing_class=tokenizer,
    )
trainer.train()
trainer.save_model("./drive/MyDrive/Results_AdaLora/Qwen/my_trained_qwen_model")
trainer.save_state()

login
data is loading


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16820 [00:00<?, ? examples/s]

Map:   0%|          | 0/2102 [00:00<?, ? examples/s]

...

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: Dali-bot/trackio-dataset
* Found existing space: https://huggingface.co/spaces/Dali-bot/trackio
* View dashboard by going to: https://Dali-bot-trackio.hf.space/


* Created new run: Dali-bot-1762796536
[Resource Log | Step 1] GPU Memory Used: 17.92 GB / 40.00 GB, CPU Usage: 12.1%, RAM Usage: 7.78 GB / 83.47 GB


Step,Training Loss
10,123.303800
20,81.876700
30,53.669000
40,50.023600
50,43.522800
60,39.156800
70,35.221700
80,31.432500
90,27.581700
100,23.681700


[Resource Log | Step 7] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.80 GB / 83.47 GB
[Resource Log | Step 13] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 7.80 GB / 83.47 GB
[Resource Log | Step 19] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.79 GB / 83.47 GB
[Resource Log | Step 25] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 7.78 GB / 83.47 GB
[Resource Log | Step 31] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.75 GB / 83.47 GB
[Resource Log | Step 37] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.79 GB / 83.47 GB
[Resource Log | Step 43] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 7.79 GB / 83.47 GB
[Resource Log | Step 49] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.7%, RAM Usage: 7.76 GB / 83.47 GB
[Resource Log | Step 55] GPU Memory Used: 17.98 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 7.78 GB / 83.47 GB
[R

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
